In [1]:
from report import *

reports_dir = '../data'
data = extract(reports_dir, reports_dir)[0]
counts = create_count_table(data)
tp = set(data[data['tp'] > 0]['test'].drop_duplicates())
# Mark tests in which value was expected to be tainted
data['true_alarm'] = data['test'].apply(lambda x: x in tp)
data.head()

Using existing aggregated datasets.


,tool,version,test,group,tp,fp,fn,status,result,trial_id,true_alarm
0,phosphor,21,[engine:junit-jupiter]/[class:edu.neu.ccs.prl....,ArrayAccess,0,0,0,vm-crash,semantic,7,True
1,phosphor,21,[engine:junit-jupiter]/[class:edu.neu.ccs.prl....,ArrayAccess,0,0,0,vm-crash,semantic,7,True
2,phosphor,21,[engine:junit-jupiter]/[class:edu.neu.ccs.prl....,ArrayAccess,0,0,0,vm-crash,semantic,7,True
3,phosphor,21,[engine:junit-jupiter]/[class:edu.neu.ccs.prl....,ArrayAccess,0,0,0,vm-crash,semantic,7,True
4,phosphor,21,[engine:junit-jupiter]/[class:edu.neu.ccs.prl....,ArrayAccess,0,0,0,vm-crash,semantic,7,True


In [2]:
# How many failures of each type did each tool have?
data[['tool', 'result']].value_counts().reset_index().sort_values(by=['tool','result'])

,tool,result,count
0,galette,success,11364
3,mirror-taint,success,4780
2,mirror-taint,tag,6584
1,phosphor,semantic,7214
4,phosphor,success,3262
5,phosphor,tag,888


In [3]:
# How failures did Phosphor have due to vm-crashes in each JDK version?
select(data, tool='phosphor', status='vm-crash')['version'].value_counts()

version
21    3451
17    3451
Name: count, dtype: int64

In [4]:
# In which groups did each tool have semantic failures other than vm-crashes?
data[(data['status'] != 'vm-crash') & (data['result'] == 'semantic')][['tool', 'group']]\
    .drop_duplicates()

,tool,group
18724,phosphor,MethodReflection
18932,phosphor,MethodHandle
19052,phosphor,ConstructorReflection
30953,phosphor,VarHandle
33664,phosphor,MethodHandleJava9


In [6]:
# In which groups did each tool have propagation failures?
x = select(data,result='tag')[['tool', 'group']]\
    .value_counts()\
    .reset_index()\
    .sort_values(by=['tool','group'])
x

,tool,group,count
5,mirror-taint,ArrayAccess,220
19,mirror-taint,ArrayLength,12
4,mirror-taint,ArrayReflection,260
22,mirror-taint,Collection,8
13,mirror-taint,ConstructorReflection,48
21,mirror-taint,Field,8
7,mirror-taint,FieldReflection,124
3,mirror-taint,JdkUnsafe,639
26,mirror-taint,Lambda,4
24,mirror-taint,Loop,4


In [11]:
# How many propagation failures were in groups related to signature polymorphism?
sig_p = ['Lambda','MethodHandle', 'MethodHandleJava9', 'RecordType', 'StringIndyConcat', 'VarHandle']
y = data[data['group'].isin(sig_p)]
select(y, result='tag')[['tool']] \
    .value_counts() \
    .reset_index() \
    .sort_values(by=['tool'])

,tool,count
0,mirror-taint,4321
1,phosphor,826


In [14]:
# What tests in the String group failed?
z = select(data, result='tag', group='String')
z['method'] = z['test'].apply(lambda x: x.replace('[engine:junit-jupiter]/[class:edu.neu.ccs.prl.galette.bench.StringITCase]/[', ''))
z[['tool', 'version', 'method', 'fn', 'fp']]

,tool,version,method,fn,fp
6796,mirror-taint,11,test-template:split(boolean)]/[test-template-i...,10,0
6798,mirror-taint,11,test-template:length(boolean)]/[test-template-...,1,0
6800,mirror-taint,11,test-template:substring(boolean)]/[test-templa...,3,0
6802,mirror-taint,11,test-template:toUpperCase(boolean)]/[test-temp...,5,0
6804,mirror-taint,11,test-template:replaceChar(boolean)]/[test-temp...,1,0
6806,mirror-taint,11,test-template:codePointAt(boolean)]/[test-temp...,1,0
6808,mirror-taint,11,test-template:toLowerCase(boolean)]/[test-temp...,5,0
6810,mirror-taint,11,test-template:chars(boolean)]/[test-template-i...,5,0
6812,mirror-taint,11,test-template:codePoints(boolean)]/[test-templ...,5,0
6814,mirror-taint,11,test-template:charAt(boolean)]/[test-template-...,1,0


In [15]:
data['group'].drop_duplicates().shape

(26,)